In [1]:
import numpy as np
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
import keras as keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from matplotlib import pyplot

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# from stats import extract_from_folder, extract_features
# chunk_size = 1000

# folder = 'data'
# subfolder = 'quasi-static'
# extension = '.npz'
# max_files = 100
# n_pca = 4

# features_quasi = extract_from_folder(folder, subfolder, extension, max_files, shuffle=False, chunk_size=chunk_size,
#                                          n_pca=n_pca).to_numpy()
# labels_quasi = np.zeros(features_quasi.shape[1])

# subfolder = 'moving'

# features_moving = extract_from_folder(folder, subfolder, extension, max_files, shuffle=False, chunk_size=chunk_size,
#                                           n_pca=n_pca).to_numpy()

# labels_moving = np.ones(features_moving.shape[1])
# print(np.hstack((features_quasi, features_moving)))
# X = np.hstack((features_quasi, features_moving)).T
# Y = np.hstack((labels_quasi, labels_moving))

# train_data, test_data, train_labels, test_labels = train_test_split(X, Y, test_size=0.2, random_state=1)

# train_data = np.reshape(train_data, (1,train_data.shape[0], train_data.shape[1]))
# test_data = np.reshape(test_data, (1,test_data.shape[0], test_data.shape[1]))
# train_labels = np.reshape(train_labels, (1,train_labels.shape[0], 1))
# test_labels = np.reshape(test_labels, (1,test_labels.shape[0], 1))
# train_labels = tf.squeeze(train_labels, axis=-1)

# print(train_data.shape)
# print(test_data.shape)
# print(train_labels.shape)
# print(test_labels.shape)

# print(train_labels[0])






In [3]:
# train_data.shape

In [4]:
from stats import extract_from_folder, extract_features
chunk_size = 1000

def load_dataset():
    folder = 'data'
    subfolder = 'quasi-static'
    extension = '.npz'
    max_files = 100
    n_pca = 4

    features_quasi = extract_from_folder(folder, subfolder, extension, max_files, shuffle=False, chunk_size=chunk_size,
                                             n_pca=n_pca).to_numpy()
    labels_quasi = np.zeros(features_quasi.shape[1])

    subfolder = 'moving'

    features_moving = extract_from_folder(folder, subfolder, extension, max_files, shuffle=False, chunk_size=chunk_size,
                                              n_pca=n_pca).to_numpy()

    labels_moving = np.ones(features_moving.shape[1])
    print(np.hstack((features_quasi, features_moving)))
    X = np.hstack((features_quasi, features_moving)).T
    Y = np.hstack((labels_quasi, labels_moving))

    train_data, test_data, train_labels, test_labels = train_test_split(X, Y, test_size=0.2, random_state=1)

    train_data = np.reshape(train_data, (1,train_data.shape[0], train_data.shape[1]))
    test_data = np.reshape(test_data, (1,test_data.shape[0], test_data.shape[1]))
    train_labels = np.reshape(train_labels, (1,train_labels.shape[0], 1))
    print(train_labels.shape)
    test_labels = np.reshape(test_labels, (1,test_labels.shape[0], 1))
    train_labels = tf.squeeze(train_labels, axis=-1)
    test_labels = tf.squeeze(test_labels, axis=-1)
    print(train_data.shape)
    print(test_data.shape)
    print(train_labels.shape)
    print(test_labels.shape)

    return train_data, train_labels, test_data, test_labels





In [5]:

def evaluate_model(train_data, train_labels, test_data, test_labels):
    n_features = train_data.shape[2]
    n_timesteps = 1
    batch_size = 70
    verbose = 0
    epochs = 15
    n_outputs = train_labels.shape[1]
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps,n_features))) #LSTM('no. of lstm blocks', input_shape(time_Steps,features))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    loss_function = keras.losses.SparseCategoricalCrossentropy()
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(train_data, train_labels, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(test_data, test_labels, batch_size=batch_size, verbose=0)
    return accuracy

In [6]:
# # fit and evaluate a model
# def evaluate_model(trainX, trainy, testX, testy):
# 	verbose, epochs, batch_size = 0, 15, 70
# # 	n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
# 	model = Sequential()
# 	model.add(LSTM(100, input_shape=(n_timesteps,n_features))) #LSTM('no. of lstm blocks', input_shape(time_Steps,features))
# 	model.add(Dropout(0.5))
# 	model.add(Dense(100, activation='relu'))
# 	model.add(Dense(n_outputs, activation='softmax'))
# 	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# 	# fit network
# 	model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
# 	# evaluate model
# 	_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
# 	return accuracy

In [7]:
# # summarize scores
# def summarize_results(scores):
# 	print(scores)
# 	m, s = mean(scores), std(scores)
# 	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [8]:
# run an experiment
def run_experiment(repeats=10):
    # load data
    train_data, train_labels, test_data, test_labels = load_dataset()
    # repeat experiment
    # 	scores = list()
    for r in range(repeats):
        score = evaluate_model(train_data, train_labels, test_data, test_labels)
        score = score * 100.0
        print('>#%d: %.3f' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [9]:
run_experiment()

100%|████████████████████████████████████████████████████████████████████████████████| 900/900 [00:12<00:00, 70.48it/s]


[[ 0.00151726  0.00108802  0.00156848 ... -0.04874717 -0.04722313
  -0.0366187 ]
 [-0.01670551 -0.02903951 -0.01962464 ... -0.02406255 -0.02137506
  -0.0730015 ]
 [-0.01954968  0.00231281  0.00111195 ...  0.00592568 -0.01300396
   0.02423217]
 [-0.01081741 -0.0165591  -0.00446242 ...  0.00769432  0.0315255
  -0.03071527]]
(1, 1440, 1)
(1, 1440, 4)
(1, 360, 4)
(1, 1440)
(1, 360)


ValueError: in user code:

    C:\Users\Swiftx\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1224 test_function  *
        return step_function(self, iterator)
    C:\Users\Swiftx\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1215 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Swiftx\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Swiftx\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Swiftx\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\Swiftx\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1208 run_st